In [1]:
# import sys
# !{sys.executable} -m pip install -r requirements.txt

In [2]:
#GENES = all low level pertubative hyper heuristics
#Swapping two randomly selected exams
#Swapping subsets of exams
#De-allocating exams
#Rescheduling exams
#Swapping timeslots of two randomly selected examinations

In [3]:
# !pip install import_ipynb

In [4]:
import sqlalchemy as db
import pandas as pd
from sqlalchemy import Column, Integer, Text, ForeignKey,String,Table, DateTime
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import random
from IPython.display import clear_output
from IPython.core.debugger import set_trace
import numpy as np
from platform import python_version

# import import_ipynb
# from finalSolution import periodHeuristics

In [5]:
def fixHardConstraints(constraints,connection):
    dates =[]
    solution_table = '(select exam_period.exam_id, exam_period.period_id, period_room.room_id from period_room inner join exam_period on period_room.period_id = exam_period.period_id) as T'
    for constraint in constraints:
        #2, EXAM_COINCIDENCE, 3
        #exam 2 and 3 should be in the same period:
        if(constraint.name == "EXAM_COINCIDENCE"):
            sql_query = db.text("SELECT COUNT(DISTINCT period_id) FROM " + solution_table + " WHERE exam_id=:idOne OR exam_id =:idTwo")
            result = connection.execute(sql_query, idOne = constraint.periods[0],idTwo = constraint.periods[1])
            for r in result:
#                 print(r[0])
                if(r[0] > 1):
                 sql_query = db.text("update exam_period set period_id = (select id from period order by random() limit 1) where exam_id in (:idOne,:idTwo)")
                 connection.execute(sql_query, idOne = constraint.periods[0],idTwo = constraint.periods[1])
        #1, EXCLUSION, 5
        #exam 1 and 5 should not be in the same period:
        if(constraint.name == "EXCLUSION"):
            sql_query = db.text("SELECT COUNT(DISTINCT period_id) FROM " + solution_table + " WHERE exam_id=:idOne OR exam_id =:idTwo")
            result = connection.execute(sql_query, idOne = constraint.periods[0],idTwo = constraint.periods[1])
            for r in result:
#                 print(r[0])
                if(r[0] < 2):
                    sql_query = db.text("update exam_period set period_id = (select id from period order by random() limit 1) where exam_id = :ID")
                    connection.execute(sql_query, ID = constraint.periods[0])
                    sql_query = db.text("update exam_period set period_id = (select id from period order by random() limit 1) where exam_id = :ID")
                    connection.execute(sql_query, ID = constraint.periods[1])
        #0, AFTER, 9
        # 0 should be timetabled after 9 
        if(constraint.name == "AFTER"):
            sql_query = db.text("SELECT exam_id, datetime FROM (SELECT exam_id, time as datetime FROM " + solution_table + " INNER JOIN period ON T.period_id = period.id) AS T WHERE exam_id =:idOne OR exam_id =:idTwo ORDER BY exam_id")
            result = connection.execute(sql_query, idOne = constraint.periods[0],idTwo = constraint.periods[1])
            for r in result:
                dates.append(r["datetime"])
            
            if(dates[0] < dates[1]):
                sql_query = db.text("update exam_period set period_id = case exam_id when :idOne then ( select period_id from exam_period where exam_id = :idTwo) when :idTwo then ( select period_id from exam_period where exam_id = :idOne) end where exam_id = :idOne or exam_id = :idTwo")
                connection.execute(sql_query, idOne = constraint.periods[0],idTwo = constraint.periods[1])
#         9, ROOM_EXCLUSIVE
#         exam 9 should be the only exam scheduled in a room
#         if(constraint.name.strip() == "ROOM_EXCLUSIVE"):
#             sql_query = db.text("select count(distinct id) as exam_count from period_room where room_id = (select room_id from ( select exam_period.exam_id, exam_period.period_id, period_room.room_id from period_room inner join exam_period on period_room.period_id = exam_period.period_id) as T where exam_id = 1) and period_id = (select period_id from ( select exam_period.exam_id, exam_period.period_id, period_room.room_id from period_room inner join exam_period on period_room.period_id = exam_period.period_id) as T where exam_id = :examId)")
#             result = connection.execute(sql_query, examId = constraint.periods[0])
#             for r in result:
#                 if(r["exam_count"] > 1):
#                     print("cant fix ROOM_EXCLUSIVE")
#                     violationCount += 1
#                     violations.append(constraint)
            

In [6]:
def pertubativeHeuristic(heuristic,period_amount,connection):
    if(heuristic == 1):
        #Swapping timeslots of two randomly selected examinations
        sql_query = db.text("begin work; savepoint pre_heuristic; update exam_period set period_id = (case when exam_id = first_exam_id then second_period_id when exam_id = second_exam_id then first_period_id else period_id end) from ( select max(first_exam_id) as first_exam_id, max(first_period_id) as first_period_id, max(second_exam_id) as second_exam_id, max(second_period_id) as second_period_id from ( select nth_value (exam_id, 1) over( order by exam_id ) first_exam_id, nth_value (period_id, 1) over( order by exam_id ) first_period_id, nth_value (exam_id, 2) over( order by exam_id ) second_exam_id, nth_value (period_id, 2) over( order by period_id ) second_period_id from ( select exam_id, period_id from exam_period order by random() limit 2) as random_exams) as T) as B;")
        connection.execute(sql_query)
    if(heuristic == 2):
        #Rescheduling exams
        sql_query = db.text("begin work; savepoint pre_heuristic; DO $$ BEGIN for i in 1..{} loop update exam_period set period_id = (select id from period order by random() limit 1) where exam_id = i; RAISE NOTICE 'exam_id: %', i; END LOOP; END $$;".format(period_amount))
        connection.execute(sql_query)
    if(heuristic == 3):
        #Reschedule one exam
        sql_query=db.text("begin work; savepoint pre_heuristic; update exam_period set period_id = (select id from period order by random() limit 1) where exam_id = (select id from exam order by random() limit 1);")
        connection.execute(sql_query)
    if(heuristic == 4):
        #Reschedule one exam
        sql_query=db.text("begin work; savepoint pre_heuristic; update exam_period set period_id = (select id from period order by random() limit 1) where exam_id = (select id from exam order by random() limit 1);")
        connection.execute(sql_query)
    if(heuristic == 5):
        #Reschedule all rooms
        sql_query=db.text("begin work; savepoint pre_heuristic; DO $$ BEGIN for i in 1..{} loop update period_room set room_id = (select id from room order by random() limit 1) where period_id = i; RAISE NOTICE 'exam_id: %', i; END LOOP; END $$;".format(period_amount))
        connection.execute(sql_query)
    if(heuristic == 6):
        #Reschedule one room
        sql_query=db.text("begin work; savepoint pre_heuristic; update period_room set room_id = (select id from room order by random() limit 1) where period_id = (select id from period order by random() limit 1);")
        connection.execute(sql_query)
    if(heuristic == 7):
        #Reschedule one room
        sql_query=db.text("begin work; savepoint pre_heuristic; update exam_period set period_id = (select id from period order by penalty limit 1) where exam_id = (select id from exam order by random() limit 1);")
        connection.execute(sql_query)
    if(heuristic == 8):
        #Reschedule one period with lowest penalty
        sql_query=db.text("begin work; savepoint pre_heuristic; update exam_period set period_id = (select id from period order by penalty limit 1) where exam_id = (select id from exam order by random() limit 1);")
        connection.execute(sql_query)
    if(heuristic == 9):
        #reschedule exam with more students to the beginning of the day
        sql_query = db.text("begin work; savepoint pre_heuristic;    update exam_period set period_id =( select id from ( select id from period order by time asc limit 5) as T order by random() limit 1) where exam_id =( select exam_id from ( select count(student_id) as students, exam_id from exam_student group by exam_id order by students desc limit 5) as T order by random() limit 1)")
        connection.execute(sql_query)

In [7]:
def getCurrentScore(softconstraints,connection):
    runningScore = 0
    solution_table = '(select exam_period.exam_id, exam_period.period_id, period_room.room_id from period_room inner join exam_period on period_room.period_id = exam_period.period_id) as S'
    for cons in softconstraints:
        if(cons.name == "TWOINAROW"):
            sql_query = db.text("select Count(student_id) as studentCount from ( select prev_exam, time, elapsed_time, exam_id from ( select time, elapsed_time, exam_id, lag(exam_id) over ( order by exam_id) prev_exam from ( select exam_id, time, time - lag(time) over ( order by time) elapsed_time from ( select exam_id, time from " + solution_table + " inner join period on S.period_id = period.id) S order by time asc) T ) T where elapsed_time between '60 MINUTES' and '90 MINUTES') as exams inner join exam_student on exams.exam_id = exam_student.exam_id and exam_student.exam_id = exams.prev_exam")
            studentCount = connection.execute(sql_query).fetchone()[0]
            runningScore += int(studentCount) * int(cons.params[0])
        
        if(cons.name == "TWOINADAY"):
            sql_query = db.text("select Count(student_id) as studentCount from ( select prev_exam, time, elapsed_time, exam_id from ( select time, elapsed_time, exam_id, lag(exam_id) over ( order by exam_id) prev_exam from ( select exam_id, time, time - lag(time) over ( order by time) elapsed_time from ( select exam_id, time from " + solution_table + " inner join period on S.period_id = period.id) T order by time asc) T ) T where elapsed_time between '1 DAYS' and '2 DAYS') as exams inner join exam_student on exams.exam_id = exam_student.exam_id and exam_student.exam_id = exams.prev_exam")
            studentCount = connection.execute(sql_query).first()[0]
            runningScore += int(int(studentCount) * cons.params[0])

        if(cons.name == "NOMIXEDDURATIONS"):
            sql_query = db.text("select sum(durations) as totalMixed from (SELECT period_id, count(distinct duration) as durations FROM ( select period_id, exam_id, duration, room_id from ( select S.exam_id, S.room_id, period_room.period_id from " + solution_table + " inner join period_room on S.period_id = period_room.period_id order by period_id) as examrooms inner join exam on examrooms.exam_id = exam.id) T GROUP BY period_id HAVING COUNT(DISTINCT duration) > 1) T")
            mixedCount = connection.execute(sql_query).fetchone()[0]
 
            if(mixedCount is not None):
                runningScore += int(mixedCount) * cons.params[0]
           
        if(cons.name == "FRONTLOAD"):
            sql_query = db.text("select count(*) as count from ( select period_id, exam_id, time from exam_period inner join period on exam_period.period_id = period.id group by period_id, time, exam_id order by time desc limit :period_amount) as b where exam_id in ( select exam_id from ( select count(student_id) students , exam_id from exam_student group by exam_id) as T order by T.students desc limit :exam_amount)");
            Count = connection.execute(sql_query, period_amount = cons.params[1],exam_amount = cons.params[0]).fetchone()[0]
            runningScore += int(Count) * cons.params[2]
    
    #calculate room penalty
    sql_query = db.text("select sum(penalty) total_penalty from period_room inner join room on period_room.room_id  = room.id")
    room_penalty = connection.execute(sql_query).fetchone()[0]
    #calculate period penalty
    sql_query = db.text("select sum(penalty) total_penalty from exam_period inner join period on exam_period.period_id  = period.id")
    period_penalty = connection.execute(sql_query).fetchone()[0]
    
    runningScore += int(room_penalty)
    runningScore += int(period_penalty)
        
    return runningScore

In [8]:
def EvaluateSolution(constraints,connection):
    violations =[]
    dates = []
    violationCount = 0
    solution_table = '(select exam_period.exam_id, exam_period.period_id, period_room.room_id from period_room inner join exam_period on period_room.period_id = exam_period.period_id) as T'
    for constraint in constraints:
        #2, EXAM_COINCIDENCE, 3
        #exam 2 and 3 should be in the same period:
        if(constraint.name == "EXAM_COINCIDENCE"):
            sql_query = db.text("SELECT COUNT(DISTINCT period_id) FROM " + solution_table + " WHERE exam_id=:idOne OR exam_id =:idTwo")
            result = connection.execute(sql_query, idOne = constraint.periods[0],idTwo = constraint.periods[1])
            for r in result:
#                 print(r[0])
                if(r[0] > 1):
                    violationCount += 1
                    violations.append(constraint)
        #1, EXCLUSION, 5
        #exam 1 and 5 should not be in the same period:
        if(constraint.name == "EXCLUSION"):
            sql_query = db.text("SELECT COUNT(DISTINCT period_id) FROM " + solution_table + " WHERE exam_id=:idOne OR exam_id =:idTwo")
            result = connection.execute(sql_query, idOne = constraint.periods[0],idTwo = constraint.periods[1])
            for r in result:
#                 print(r[0])
                if(r[0] < 2):
                    violationCount += 1
                    violations.append(constraint)
        #0, AFTER, 9
        # 0 should be timetabled after 9 
        if(constraint.name == "AFTER"):
            sql_query = db.text("SELECT exam_id, datetime FROM (SELECT exam_id, time as datetime FROM " + solution_table + " INNER JOIN period ON T.period_id = period.id) AS T WHERE exam_id =:idOne OR exam_id =:idTwo ORDER BY exam_id")
            result = connection.execute(sql_query, idOne = constraint.periods[0],idTwo = constraint.periods[1])
            for r in result:
                dates.append(r["datetime"])
            
            if(dates[0] < dates[1]):
                violationCount += 1;
            result_as_list = result.fetchall()
        #9, ROOM_EXCLUSIVE
        # exam 9 should be the only exam scheduled in a room
#         if(constraint.name.strip() == "ROOM_EXCLUSIVE"):
#             sql_query = db.text("select count(distinct id) as exam_count from period_room where room_id = (select room_id from ( select exam_period.exam_id, exam_period.period_id, period_room.room_id from period_room inner join exam_period on period_room.period_id = exam_period.period_id) as T where exam_id = 1) and period_id = (select period_id from ( select exam_period.exam_id, exam_period.period_id, period_room.room_id from period_room inner join exam_period on period_room.period_id = exam_period.period_id) as T where exam_id = :examId)")
#             result = connection.execute(sql_query, examId = constraint.periods[0])
#             for r in result:
#                 if(r["exam_count"] > 1):
#                     violationCount += 1
#                     violations.append(constraint)
            
    return violationCount;

In [9]:
def clearPeriodTable(connection):
    sql_query = db.text("TRUNCATE exam_period RESTART IDENTITY")
    connection.execute(sql_query)
    

In [10]:
def generateSolution(examH,periodH,connection,session,examRows,periodRows):
    exam = None
    period = None
    clearPeriodTable(connection)
    exam,period = heuristics(examH,periodH,examRows,periodRows,connection,session)
    createSolution(examRows,periodRows,connection,session)
    return exam,period

In [11]:
def createSolution(examRows,periodRows, connection,session):
    heuristics(None,None,examRows,periodRows,connection,session)

In [12]:
def heuristics(examH,periodH,examRows,periodRows,connection,session):
    allocated_exams = [0]
    allocated_periods = [0]
    exam_heuristic = random.randint(1,2);
    period_heuristic = random.randint(1,4);
    while len(allocated_exams)-1 < len(examRows):
        examHeuristics(allocated_periods,allocated_exams, exam_heuristic,period_heuristic,examH,periodH,examRows,periodRows,connection,session)
        
    return exam_heuristic,period_heuristic

In [13]:
def periodHeuristics(allocated_periods,period_heuristic,connection):
    periods = ','.join([str(elem) for elem in allocated_periods]) 
    if(period_heuristic == 1):
        sql_query = db.text("select id from period order by penalty limit 1")
        period = connection.execute(sql_query).fetchone()[0]
        allocated_periods.append(period)
    if(period_heuristic == 2):
        sql_query = db.text("select id from period order by time asc limit 1")
        period = connection.execute(sql_query).fetchone()[0]
        allocated_periods.append(period)
    if(period_heuristic == 3):
        sql_query = db.text("select id from period order by duration asc limit 1")
        period = connection.execute(sql_query).fetchone()[0]
        allocated_periods.append(period)     
    if(period_heuristic == 4):
        sql_query = db.text("select id from period order by random() asc limit 1")
        period = connection.execute(sql_query).fetchone()[0]
        allocated_periods.append(period)
    return period

In [14]:
def examHeuristics(allocated_exams,allocated_periods,exam_heuristic, period_heuristic, examH,periodH,examRows,periodRows,connection,session):
    allocated = ','.join([str(elem) for elem in allocated_exams])
    periodId = periodHeuristics(allocated_periods,period_heuristic,connection)
    if(exam_heuristic == 1 and examH != 1):
        #schedule exam with most clashes first
        sql_query = db.text("select exam_id from staticHeuristics  where exam_id not in (" + allocated + ") order by exam_count desc limit 1")
        exam = connection.execute(sql_query).fetchone()
        if exam is None:
            return None
        exam = exam[0]
        examRows[exam-1].periods.append(periodRows[periodId -1])
        session.add(examRows[exam-1])
        session.commit()
        allocated_exams.append(exam)
    if(exam_heuristic == 2 and examH != 2):
        #schedule exam with most students first
        sql_query = db.text("select exam_id from staticheuristics where exam_id not in (" + allocated + ")  order by student_count desc limit 1")
        exam = connection.execute(sql_query).fetchone()
        if exam is None:
            return None
        exam = exam[0]
        examRows[exam-1].periods.append(periodRows[periodId -1])
        session.add(examRows[exam-1])
        session.commit()
        allocated_exams.append(exam)

In [15]:
def genInitialSolution(connection,session,constraints,examRows,periodRows,file):
    hardConstraintViolations = EvaluateSolution(constraints,connection)
    iteration = 0
    scores = []
#     examHeuristic = 0
#     periodHeuristic = 0
    examH = None
    periodH = None
    while True:
        random.seed(random.randint(3, 9))
        print('amount of hard constraint violations', hardConstraintViolations)
        print("iteration:",iteration)
        if hardConstraintViolations == 0: 
            break

        fixHardConstraints(constraints,connection)
        
        
#         examHeuristic,periodHeuristic = generateSolution(examH, periodH, connection,session,examRows,periodRows) 

        hardViolationCount = EvaluateSolution(constraints,connection)  

        print("neighbour hard violation count:",hardConstraintViolations)
#         if hardViolationCount > 0:
# #             examH = examHeuristic
# #             periodH = periodHeuristic
        if hardViolationCount < hardConstraintViolations : 
            hardConstraintViolations = hardViolationCount
    #     clear_output(wait=True)
        iteration += 1
        clear_output(wait=True)

In [16]:
class Constraint:
    def __init__(self,ctype,name,periods):
        self.ctype = ctype
        self.name = name
        self.periods = periods

In [17]:
def generateStatisHeurisitcs(connection):
    sql_query = db.text("create or replace view staticHeuristics as  select a.exam_id, b.student_id, b.exam_count, a.student_count from ( select exam_id, count(student_id) as student_count from exam_student group by exam_id) as a, ( select student_id , count(exam_id) as exam_count from exam_student group by student_id) as b order by b.exam_count desc")
    connection.execute(sql_query) 

In [18]:
def populateDB(engine,session,Base,connection,sample):
    association_table = Table('exam_student', Base.metadata,
    Column('id',db.Integer, primary_key=True),
    Column('exam_id', Integer, ForeignKey('exam.id',ondelete="cascade")),
    Column('student_id', Integer, ForeignKey('student.id',ondelete="cascade"))
    )

    association_two = Table('exam_period', Base.metadata,
        Column('id',db.Integer, primary_key=True),
        Column('exam_id', Integer, ForeignKey('exam.id',ondelete="cascade")),
        Column('period_id', Integer, ForeignKey('period.id',ondelete="cascade"))
    )

    association_three = Table('period_room', Base.metadata,
        Column('id',db.Integer, primary_key=True),
        Column('period_id', Integer, ForeignKey('period.id',ondelete="cascade")),
        Column('room_id', Integer, ForeignKey('room.id',ondelete="cascade"))
    )

    class Exam(Base):
        __tablename__ = 'exam'

        id = Column(Integer, primary_key = True)
        duration = Column(Integer)
        students = relationship(
        "Student",
        secondary=association_table,
        back_populates="exams",
        cascade="all, delete", passive_deletes=True)
        periods = relationship(
        "Period",
        secondary=association_two,
        back_populates="exams",
        cascade="all, delete", passive_deletes=True)



    class Student(Base):
        __tablename__ = 'student'

        id = Column(Integer, primary_key = True)
        #    examid = Column(Integer, ForeignKey('exams.id'))
        number = Column(Integer)
        #    exams = relationship(Exam,secondary='link')
        exams = relationship(
            "Exam",
            secondary=association_table,
            back_populates="students",
            cascade="all, delete", passive_deletes=True)
        
    class Constraint:
        def __init__(self,ctype,name,periods):
            self.ctype = ctype
            self.name = name
            self.periods = periods
            
    class SoftConstraint:
        def __init__(self,name,params):
            self.name = name
            self.params = params

    class Room(Base):
        __tablename__ = 'room'

        id = Column(Integer, primary_key = True)
        capacity = Column(Integer)
        penalty = Column(Integer)
        periods = relationship(
            "Period",
            secondary=association_three,
            back_populates="rooms",
            cascade="all, delete", passive_deletes=True)


    class Period(Base):
        __tablename__ = 'period'
        id = Column(Integer, primary_key = True)
        time = Column(DateTime)
        duration = Column(Integer)
        penalty = Column(Integer)
        exams = relationship(
            "Exam",
            secondary=association_two,
            back_populates="periods",
            cascade="all, delete", passive_deletes=True)
        rooms = relationship(
            "Room",
            secondary=association_three,
            back_populates="periods",
            cascade="all, delete", passive_deletes=True)
    Base.metadata.create_all(engine)
    
    for tbl in reversed(Base.metadata.sorted_tables):
        print(tbl.name)
        truncate_table = db.text("TRUNCATE TABLE " + tbl.name + " RESTART IDENTITY CASCADE")
        engine.execute(truncate_table)
    
    
    roomRows = []

    with open(sample) as f:
         for line in f:
            if "Exams" in line:
                line = f.readline()
                lineType = "Exams"
            if "Periods" in line:
                line = f.readline()
                lineType = "Periods"
            if "Rooms" in line:
                line = f.readline()
                lineType = "Rooms"
            if "PeriodHardConstraints" in line:
                lineType = "PeriodHardConstraints"
                line = f.readline()
            if "RoomHardConstraints" in line:
                line = f.readline()
                lineType = "RoomHardConstraints"
            if "InstitutionalWeightings" in line:
                line = f.readline()
                lineType = "InstitutionalWeightings"

            if(lineType == 'Rooms'):
                arr = line.split(',')
                r1 = Room(capacity = arr[0],penalty = arr[1])
                roomRows.append(r1)

    session.add_all(roomRows)
    session.commit()
    
    examRows =[]
    periodRows = []
    studentRows = []
    # roomRows = []
    students = []
    constraints = []
    softconstraints =[]
    examcount = 0
    periodcount = 0
    roomcount = 0
    lineType = ""
    with open(sample) as f:
         for line in f:
            if "Exams" in line:
                line = f.readline()
                lineType = "Exams"
            if "Periods" in line:
                line = f.readline()
                lineType = "Periods"
            if "Rooms" in line:
                line = f.readline()
                lineType = "Rooms"
            if "PeriodHardConstraints" in line:
                lineType = "PeriodHardConstraints"
                line = f.readline()
            if "RoomHardConstraints" in line:
                line = f.readline()
                lineType = "RoomHardConstraints"
            if "InstitutionalWeightings" in line:
                line = f.readline()
                lineType = "InstitutionalWeightings"


            if(lineType == 'Periods'):
                arr = line.split(',')
                dateTime = arr[0] + arr[1]
                p1 = Period(time = datetime.strptime(dateTime,'%d:%m:%Y %H:%M:%S'),duration = arr[2],penalty = arr[3])
                periodRows.append(p1)
                periodRows[periodcount].rooms.append(roomRows[random.randint(0,len(roomRows)-1)])
                periodcount += 1

            if(lineType == 'PeriodHardConstraints'):
                arr = line.split(',')
                print("period",arr)
                c1 = Constraint("period",name =arr[1].strip(),periods = [int(arr[0].strip()) + 1, int(arr[2].strip()) + 1])
                constraints.append(c1)

            if(lineType == 'RoomHardConstraints'):
                arr = line.split(',')
                print("room",arr)
                if len(arr) > 1 :
                    c1 = Constraint("room",name = arr[1].strip(),periods = [int(arr[0].strip())+1])
                    constraints.append(c1)
                else:
                    c1 = Constraint("room",name = arr[0].strip(),periods = None)
                    constraints.append(c1)
            if(lineType == "InstitutionalWeightings"):
                arr = [x.strip() for x in line.split(',')]
                print(arr)
                sc1 = SoftConstraint(name = arr[0],params = [int(i) for i in arr[1:]])
                softconstraints.append(sc1)

            if(lineType == 'Exams'):
                arr = line.split(',')
                examRows.append(Exam(duration = int(arr[0])))
               

                for no in arr[1:]:
                    no = no.strip()
                    no = int(no)
                    if no not in students:
                        students.append(int(no))
                        tempStudent = Student(number = int(no))
                        studentRows.append(tempStudent)
                    if no in students:
                        studentRows[students.index(no)].exams.append(examRows[examcount])
    #                     examRows[examcount].students.append(studentRows[students.index(no)])
                examcount += 1
        
    session.add_all(examRows)
    session.add_all(periodRows)
    session.add_all(studentRows)
    session.add_all(roomRows)
    session.commit()
    
    
    
    generateStatisHeurisitcs(connection)
    
    
    createSolution(examRows,periodRows,connection,session)
    
    return softconstraints,constraints,examRows,periodRows,periodcount